In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Opening Files and Merging

In [ ]:
prescription = pd.read_pickle('../nosakh_ARASH.pkl')
# Parsing dates
prescription['date'] = pd.to_datetime(prescription['date'])
prescription

In [3]:
prescription.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60800 entries, 8488 to 60885400
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            60800 non-null  datetime64[ns]
 1   member          60800 non-null  int64         
 2   service         60800 non-null  int32         
 3   quantity        60800 non-null  int16         
 4   physician       60800 non-null  int32         
 5   institute       60800 non-null  int32         
 6   claimed         60800 non-null  int32         
 7   deduction       60800 non-null  int32         
 8   insurance_paid  60800 non-null  int32         
 9   patient_paid    60800 non-null  int32         
 10  fund            60800 non-null  int8          
 11  province        60800 non-null  int8          
 12  prescription    60800 non-null  int32         
dtypes: datetime64[ns](1), int16(1), int32(8), int64(1), int8(2)
memory usage: 3.5 MB


In [ ]:
physician = pd.read_pickle('../physician.pkl')
physician

In [5]:
physician.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179532 entries, 0 to 179531
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   physician     179532 non-null  int32 
 1   specialty_fa  179532 non-null  object
 2   specialty_en  179532 non-null  object
dtypes: int32(1), object(2)
memory usage: 3.4+ MB


In [ ]:
# Merging specialty_en to prescription
prescription_m_1 = prescription.merge(
    physician[['physician', 'specialty_en']],
    how='left',
    on='physician'
)
prescription_m_1

In [ ]:
profile = pd.read_pickle('../profile_ARASH.pkl')
# Parsing birthdate
profile['birthdate'] = pd.to_datetime(profile['birthdate'])
profile

In [8]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 84 to 6350019
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   member       1000 non-null   int64         
 1   birthdate    1000 non-null   datetime64[ns]
 2   gender       1000 non-null   int8          
 3   disease      2 non-null      float64       
 4   fund         1000 non-null   float64       
 5   responsible  962 non-null    float64       
 6   source       1000 non-null   int8          
dtypes: datetime64[ns](1), float64(3), int64(1), int8(2)
memory usage: 48.8 KB


In [ ]:
# Merging birthdate and gender to prescription
prescription_m_2 = prescription_m_1.merge(
    profile[['member', 'birthdate', 'gender']],
    how='left',
    on='member'
)
prescription_m_2

### Processing service before merging

In [ ]:
service = pd.read_parquet('../service.parquet')
service

In [11]:
service.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3842 entries, 0 to 3841
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   service       3842 non-null   int32  
 1   atc           3842 non-null   object 
 2   atc_1         3842 non-null   object 
 3   atc_2         3585 non-null   object 
 4   atc_3         3578 non-null   object 
 5   atc_4         3540 non-null   object 
 6   atc_5         3368 non-null   object 
 7   atc_name      3313 non-null   object 
 8   ihio_name     3842 non-null   object 
 9   generic_name  1708 non-null   object 
 10  route         1707 non-null   object 
 11  form          1707 non-null   object 
 12  str           1697 non-null   object 
 13  measure       1706 non-null   object 
 14  ddd           929 non-null    float64
dtypes: float64(1), int32(1), object(13)
memory usage: 435.4+ KB


In [12]:
# Labeling DM drugs and antibiotics in service for future merging with
# prescription_m_2
dm_atc_3 = ['A10A', 'A10B']
ab_atc_2 = ['J01', 'J02', 'J04', 'J05']
service['dm'] = service['atc_3'].isin(dm_atc_3)
service['ab'] = service['atc_2'].isin(ab_atc_2)

In [ ]:
service

In [14]:
service.loc[service['dm']].shape

(52, 17)

In [15]:
service.loc[service['ab']].shape

(304, 17)

In [ ]:
# Merging (atc, dm, ab) from service to prescription_m_2 on service
prescription_m_3 = prescription_m_2.merge(
    service[['service', 'atc', 'dm', 'ab']],
    how='left',
    on='service'
)
prescription_m_3

In [17]:
prescription_m_3.loc[prescription_m_3['dm'] | prescription_m_3['ab']]['member'].nunique()

1000

In [18]:
absent_physician = ~ prescription_m_3['physician'].isin(physician['physician'])
absent_physician.sum()

1083

In [19]:
prescription_m_3.loc[absent_physician, 'physician'].nunique()

177

There are more physicians in the prescription data than in the physician dataframe.

### Merging DDD data from ab_service_final

In [ ]:
ab_service_final = pd.read_csv('../ab_service_final.csv', index_col=0)
ab_service_final['service'] = ab_service_final.index
ab_service_final

In [ ]:
# Merging (calc_ddd, recheck) from ab_service_final to prescription_m_3
# on service
prescription_m_4 = prescription_m_3.merge(
    ab_service_final[['service', 'calc_ddd', 'recheck']],
    how='left',
    on='service'
)
prescription_m_4

In [22]:
temp = prescription_m_4['claimed'] == prescription_m_4['insurance_paid']

In [23]:
prescription_m_4.shape[0] - temp.sum()

969

The claimed and insurance_paid columns aren't the same.

In [24]:
prescription_m_4.columns

Index(['date', 'member', 'service', 'quantity', 'physician', 'institute',
       'claimed', 'deduction', 'insurance_paid', 'patient_paid', 'fund',
       'province', 'prescription', 'specialty_en', 'birthdate', 'gender',
       'atc', 'dm', 'ab', 'calc_ddd', 'recheck'],
      dtype='object')

In [25]:
prescription_m_4['deduction'].nunique()

322

In [26]:
prescription_m_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60800 entries, 0 to 60799
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            60800 non-null  datetime64[ns]
 1   member          60800 non-null  int64         
 2   service         60800 non-null  int32         
 3   quantity        60800 non-null  int16         
 4   physician       60800 non-null  int32         
 5   institute       60800 non-null  int32         
 6   claimed         60800 non-null  int32         
 7   deduction       60800 non-null  int32         
 8   insurance_paid  60800 non-null  int32         
 9   patient_paid    60800 non-null  int32         
 10  fund            60800 non-null  int8          
 11  province        60800 non-null  int8          
 12  prescription    60800 non-null  int32         
 13  specialty_en    59717 non-null  object        
 14  birthdate       60800 non-null  datetime64[ns]
 15  ge

### Removing unwanted columns

In [27]:
unwanted = ['physician', 'institute', 'claimed', 'fund']
prescription_m_final = prescription_m_4.drop(unwanted, axis=1)

In [ ]:
prescription_m_final

In [29]:
prescription_m_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60800 entries, 0 to 60799
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            60800 non-null  datetime64[ns]
 1   member          60800 non-null  int64         
 2   service         60800 non-null  int32         
 3   quantity        60800 non-null  int16         
 4   deduction       60800 non-null  int32         
 5   insurance_paid  60800 non-null  int32         
 6   patient_paid    60800 non-null  int32         
 7   province        60800 non-null  int8          
 8   prescription    60800 non-null  int32         
 9   specialty_en    59717 non-null  object        
 10  birthdate       60800 non-null  datetime64[ns]
 11  gender          60800 non-null  int8          
 12  atc             60800 non-null  object        
 13  dm              60800 non-null  bool          
 14  ab              60800 non-null  bool          
 15  ca

## Saving the final dataframe
It contains prescription data except ['physician', 'institute', 'claimed', 'fund'] columns. Plus (specialty_en) from physician and (birthdate, gender) from profile and (atc, dm, ab) from service and mask on that and (calc_ddd, recheck) from ab_service_final.

In [30]:
# prescription_m_final.to_csv('./prescription_merged_final.csv')

## Sandbox

In [ ]:
from statistics import mode

temp = prescription.groupby(['member', 'prescription'])['province'].apply(lambda g: g.iloc[0])
temp = pd.DataFrame(temp)
temp

In [ ]:
# temp['prescription'] = temp.index.get_level_values('prescription')
temp.groupby('member')['province'].apply(mode)

In [ ]:
temp

In [86]:
bins = pd.IntervalIndex.from_tuples([(1, 100), (101, 532), (533, 1000)], 'both')
bins

IntervalIndex([[1, 100], [101, 532], [533, 1000]], dtype='interval[int64, both]')

In [ ]:
pd.cut(prescription['quantity'], bins=bins)

In [ ]:
prescription['quantity_cat'] = pd.cut(prescription['quantity'], bins=bins)
prescription['quantity_cat']

In [91]:
pd.IntervalIndex.from_tuples([
    (18, 24),
    (25, 34),
    (35, 44),
    (45, 54),
    (55, 64),
    (65, 74),
    (75, 95)
],
    'both'
)

IntervalIndex([[18, 24], [25, 34], [35, 44], [45, 54], [55, 64], [65, 74], [75, 95]], dtype='interval[int64, both]')